In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# read the geopandas data frame
gdf = gpd.read_file('../data/raw/cb_2018_us_county_500k')

In [3]:
gdf.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,"POLYGON ((-89.18137 37.04630, -89.17938 37.053..."
1,21,017,00516855,0500000US21017,21017,Bourbon,06,750439351,4829777,"POLYGON ((-84.44266 38.28324, -84.44114 38.283..."
2,21,031,00516862,0500000US21031,21031,Butler,06,1103571974,13943044,"POLYGON ((-86.94486 37.07341, -86.94346 37.074..."
3,21,065,00516879,0500000US21065,21065,Estill,06,655509930,6516335,"POLYGON ((-84.12662 37.64540, -84.12483 37.646..."
4,21,069,00516881,0500000US21069,21069,Fleming,06,902727151,7182793,"POLYGON ((-83.98428 38.44549, -83.98246 38.450..."


In [4]:
gdf = gdf.to_crs(epsg=5070)
gdf['centroid'] = gdf.centroid

In [5]:
gdf.head()

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,centroid
0,21,007,00516850,0500000US21007,21007,Ballard,06,639387454,69473325,"POLYGON ((600246.776 1577487.744, 600367.325 1...",POINT (616150.239 1580009.179)
1,21,017,00516855,0500000US21017,21017,Bourbon,06,750439351,4829777,"POLYGON ((998945.055 1755388.822, 999069.425 1...",POINT (1019399.205 1749271.850)
2,21,031,00516862,0500000US21031,21031,Butler,06,1103571974,13943044,"POLYGON ((796316.473 1596948.317, 796423.808 1...",POINT (817913.675 1614111.473)
3,21,065,00516879,0500000US21065,21065,Estill,06,655509930,6516335,"POLYGON ((1035030.065 1687824.231, 1035174.629...",POINT (1048438.491 1694830.543)
4,21,069,00516881,0500000US21069,21069,Fleming,06,902727151,7182793,"POLYGON ((1036063.852 1778343.787, 1036156.023...",POINT (1061825.361 1773137.784)


In [6]:
# columns I want
columns = ['STATEFP', 'COUNTYFP', 'GEOID', 'NAME', 'geometry', 'centroid']
gdf = gdf[columns]

In [7]:
# change names to columns
column_names = ['state_fips', 'county_fips', 'geoid', 'name', 'geometry', 'centroid']
gdf.columns = column_names

In [8]:
gdf.head()

,state_fips,county_fips,geoid,name,geometry,centroid
0,21,007,21007,Ballard,"POLYGON ((600246.776 1577487.744, 600367.325 1...",POINT (616150.239 1580009.179)
1,21,017,21017,Bourbon,"POLYGON ((998945.055 1755388.822, 999069.425 1...",POINT (1019399.205 1749271.850)
2,21,031,21031,Butler,"POLYGON ((796316.473 1596948.317, 796423.808 1...",POINT (817913.675 1614111.473)
3,21,065,21065,Estill,"POLYGON ((1035030.065 1687824.231, 1035174.629...",POINT (1048438.491 1694830.543)
4,21,069,21069,Fleming,"POLYGON ((1036063.852 1778343.787, 1036156.023...",POINT (1061825.361 1773137.784)


In [9]:
# check the number of counties for alaska. I should drop this along with hawaii: 02 and 15
gdf = gdf[-gdf['state_fips'].isin(['02', '15', '60', '66', '66', '69', '72', '78'])]

In [10]:
# create a dataframe that is the cross product of counties
st1 = pd.DataFrame(gdf[['geoid', 'state_fips', 'centroid']])
st2 = pd.DataFrame(gdf[['geoid', 'state_fips', 'centroid']])
cross_states = st1.merge(st2, how = 'cross')
cross_states.columns = ['county1', 'state1', 'centroid1', 'county2', 'state2', 'centroid2']
cross_states = cross_states.loc[cross_states['state1'] != cross_states['state2'],]

In [11]:
cross_states.info(show_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9357622 entries, 11 to 9659662
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype   
---  ------     --------------    -----   
 0   county1    9357622 non-null  object  
 1   state1     9357622 non-null  object  
 2   centroid1  9357622 non-null  geometry
 3   county2    9357622 non-null  object  
 4   state2     9357622 non-null  object  
 5   centroid2  9357622 non-null  geometry
dtypes: geometry(2), object(4)
memory usage: 499.8+ MB


In [12]:
cross_states.head()

,county1,state1,centroid1,county2,state2,centroid2
11,21007,21,POINT (616150.239 1580009.179),17091,17,POINT (676922.975 2043418.015)
12,21007,21,POINT (616150.239 1580009.179),17187,17,POINT (450064.450 1994793.779)
13,21007,21,POINT (616150.239 1580009.179),17197,17,POINT (664354.473 2076858.957)
14,21007,21,POINT (616150.239 1580009.179),18027,18,POINT (768037.059 1777644.290)
15,21007,21,POINT (616150.239 1580009.179),18061,18,POINT (856316.386 1729277.725)


In [13]:
# calculate the distance between the two centroids
cross_one = gpd.GeoDataFrame(cross_states[['county1', 'county2', 'centroid1']], geometry='centroid1')
cross_two = gpd.GeoDataFrame(cross_states[['county1', 'county2', 'centroid2']], geometry='centroid2')
cross_states['distance'] = cross_one.distance(cross_two, align = False)

In [14]:
cross_states

,county1,state1,centroid1,county2,state2,centroid2,distance
11,21007,21,POINT (616150.239 1580009.179),17091,17,POINT (676922.975 2043418.015),4.673768e+05
12,21007,21,POINT (616150.239 1580009.179),17187,17,POINT (450064.450 1994793.779),4.468006e+05
13,21007,21,POINT (616150.239 1580009.179),17197,17,POINT (664354.473 2076858.957),4.991827e+05
14,21007,21,POINT (616150.239 1580009.179),18027,18,POINT (768037.059 1777644.290),2.492574e+05
15,21007,21,POINT (616150.239 1580009.179),18061,18,POINT (856316.386 1729277.725),2.827735e+05
...,...,...,...,...,...,...,...
9659658,26139,26,POINT (810188.293 2260883.501),45033,45,POINT (1508681.978 1391009.110),1.115605e+06
9659659,26139,26,POINT (810188.293 2260883.501),31073,31,POINT (-321757.388 1951127.017),1.173563e+06
9659660,26139,26,POINT (810188.293 2260883.501),39075,39,POINT (1177076.636 2037152.284),4.297240e+05
9659661,26139,26,POINT (810188.293 2260883.501),48171,48,POINT (-282857.149 809077.443),1.817275e+06


In [15]:
print(cross_states['state1'].unique())

['21' '17' '18' '01' '05' '06' '08' '09' '11' '12' '13' '16' '19' '20'
 '48' '29' '30' '31' '53' '22' '23' '24' '34' '35' '36' '37' '38' '39'
 '40' '49' '41' '42' '45' '46' '47' '25' '26' '51' '27' '28' '32' '33'
 '04' '54' '55' '56' '50' '10' '44']


In [16]:
cross_states = cross_states.sort_values(['county1', 'county2'])
cross_states

,county1,state1,centroid1,county2,state2,centroid2,distance
50544,01001,01,POINT (872696.731 1094407.437),04001,04,POINT (-1210747.610 1455917.555),2.114576e+06
51228,01001,01,POINT (872696.731 1094407.437),04003,04,POINT (-1290624.675 1071883.174),2.163439e+06
50195,01001,01,POINT (872696.731 1094407.437),04005,04,POINT (-1405355.862 1537180.303),2.320683e+06
50433,01001,01,POINT (872696.731 1094407.437),04007,04,POINT (-1355983.747 1298428.105),2.237999e+06
50196,01001,01,POINT (872696.731 1094407.437),04009,04,POINT (-1286203.031 1189862.357),2.161009e+06
...,...,...,...,...,...,...,...
4633361,56045,56,POINT (-685368.123 2347405.831),55133,55,POINT (623180.623 2249972.178),1.312171e+06
4633187,56045,56,POINT (-685368.123 2347405.831),55135,55,POINT (557824.874 2407395.704),1.244640e+06
4633608,56045,56,POINT (-685368.123 2347405.831),55137,55,POINT (538645.224 2366098.422),1.224156e+06
4633788,56045,56,POINT (-685368.123 2347405.831),55139,55,POINT (586626.277 2364702.872),1.272112e+06


In [17]:
print(cross_states['county2'].unique())

['04001' '04003' '04005' ... '01129' '01131' '01133']
